# itch_trade_identification_pandas_exp

#### Juan Camilo Henao Londono - 02.03.2019
#### AG Guhr - Universitaet Duisburg-Essen

I implement a new approach to find all the trade signs in the market time as the previous implementation was having problems identifying the trades and the corresponding trade signs.

I load the dataset as a pandas dataframe and divide it in two dataframes, one for the limit orders and another for the trade orders. As the limit order is a huge amount of data, I reduced to only take care of the data related with the trade orders.

I identified in each step of the `for loop` the limit order associated with the corresponding trade order using the order id. With this I save the time, trade sign, volume and price.

With the volumes there is a special case if there is a full or partial transaction. So it is necessary to organize the data after this happens.

When a full transaction is made, the order is change to zero to avoid repeating the value.

In [1]:
import csv
import gzip
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import datetime

%matplotlib inline

In [2]:
tickers = ['AAPL', 'AAPL', 'GS', 'GS', 'XOM', 'XOM']
year = '2008'
months = ['01', '06', '10', '12', '02', '08']
days = ['07', '02', '07', '10', '11', '04']

ticker = 'AAPL'
year = '2008'
month = '01'
day = '07'

In [3]:
time_01 = datetime.datetime.now()

# Load full data using cols with values time, order, type, shares and price
data = pd.read_csv(gzip.open('../ITCH_{1}/{1}{2}{3}_{0}.csv.gz'.format(ticker, year, month, day), 'rt'),
                   usecols=(0,2,3,4,5), 
                   #dtype=([('Time', 'uint32'), ('Order', 'uint64'), ('T', 'str'),
                   #                             ('Shares', 'uint16'), ('Price', 'float32')]))
                   #dtype=(np.uint32, np.uint64, str, np.uint16, np.float32))
                   dtype={'Time': 'uint32', 'Order': 'uint64', 'T': str,
                                                'Shares': 'uint16', 'Price': 'float32'})
                   
data['Price'] = data['Price'] / 10000

market_time = (data['Time'] / 3600 / 1000 >= 9.666666) & \
                        (data['Time'] / 3600 / 1000 < 15.833333)
    
data_market_time = data[market_time]
identified_limit_orders = len(data_market_time[data_market_time['T'] == 'B']) \
                    + len(data_market_time[data_market_time['T'] == 'S'])
identified_trades = len(data_market_time[data_market_time['T'] == 'E']) \
                    + len(data_market_time[data_market_time['T'] == 'F'])

print('The stock {} on {}.{}.{} has {} limit orders'.format(ticker, year, month, day, identified_limit_orders))
print('The stock {} on {}.{}.{} has {} trades'.format(ticker, year, month, day, identified_trades))

# Select only trade orders
trade_pos = np.array(data['T'] == 'E') + np.array(data['T'] == 'F')
trade_data = data[trade_pos].copy()
# The index are reseted because they had the previous values
# of the original data
trade_data = trade_data.reset_index()
print(trade_data.dtypes)

# Select only the limit orders
limit_pos = np.array(data['T'] == 'B') + np.array(data['T'] == 'S')
limit_data = data[limit_pos].copy()
# Reduce the values to only the ones that have the same order number
# as trade orders
limit_data = limit_data[limit_data.Order.isin(trade_data['Order'])]
# The index are reseted because they had the previous values
# of the original data
limit_data = limit_data.reset_index()

# Arrays to store the info of the identified trades
length_trades = len(trade_data)
trade_times = np.zeros(length_trades, dtype='uint32')
trade_signs = np.zeros(length_trades)
trade_volumes = np.zeros(length_trades, dtype='uint16')
trade_price = np.zeros(length_trades)

time_02 = datetime.datetime.now()
print(time_02 - time_01)

The stock AAPL on 2008.01.07 has 745020 limit orders
The stock AAPL on 2008.01.07 has 120287 trades
index       int64
Time       uint32
Order      uint64
T          object
Shares     uint16
Price     float32
dtype: object
0:00:02.012771


In [4]:
time_01 = datetime.datetime.now()

for idx in range(len(trade_data)):
    # limit orders that have the same order as the trade order   
    limit_val = limit_data[trade_data['Order'].iloc[idx] == limit_data['Order']]

    # Save values that are independent of the type

    # Time when the trade was done (Trade data)
    trade_times[idx] = trade_data['Time'].iloc[idx]
    # Price of the trade (Limit data)
    trade_price[idx] = limit_val['Price'].iloc[0]

    # Trade sign identification

    trade = limit_val['T'].iloc[0]

    if (trade == 'S'):
        trade_signs[idx] = 1.
    else:
        trade_signs[idx] = -1.

    # The volume depends on the trade type. If it is F the
    # value is taken from the limit data and the order number
    # is deleted from the data. If it is E the
    # value is taken from the trade data and then the
    # value of the volume in the limit data must be 
    # reduced with the value of the trade data

    index = limit_data.index[limit_data['Order'] == limit_val['Order'].iloc[0]]
    volume_type = trade_data['T'].iloc[idx]

    if (volume_type == 'F'):

        trade_volumes[idx] = limit_val['Shares'].iloc[0]
        limit_data.at[index, 'Order'] = 0

    else:

        trade_volumes[idx] = trade_data['Shares'].iloc[idx]
        diff_volumes = limit_val['Shares'].iloc[0] - trade_data['Shares'].iloc[idx]
        assert diff_volumes > 0

        limit_data.at[index, 'Shares'] = diff_volumes

assert not sum(trade_signs == 0)

time_02 = datetime.datetime.now()
print(time_02 - time_01)

0:10:44.403699


In [5]:
market_time = (trade_times / 3600 / 1000 >= 9.666666) & \
                    (trade_times / 3600 / 1000 < 15.833333)
    
trade_times_market = trade_times[market_time] 
trade_signs_market = trade_signs[market_time]
trade_volumes_market = trade_volumes[market_time]
trade_price_market = trade_price[market_time]

In [6]:
print('Identified trades =', len(trade_times_market))

Identified trades = 120287
